In [ ]:
import string
import re
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import spacy
import time
from textblob import TextBlob
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from bs4 import BeautifulSoup
import unicodedata

from numpy import array, argmax, random, take
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, RepeatVector, Bidirectional, SpatialDropout1D, Dropout
#from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from tensorflow.keras import optimizers, models
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ReviewsAndResponses-50K.csv", index_col=None)
df.shape

(50000, 2)

In [ ]:
df_sample = df.iloc[30000:50000]
df_sample.shape

(20000, 2)

## Text Pre processing

In [ ]:
def expand_words(x):
	contractions = {"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how does","i'd": "i would","i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have","isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is","should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there would","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not"," u ": " you "," ur ": " your "," n ": " and ","won't": "would not","dis": "this","bak": "back","brng": "bring"
	}

	if type(x) is str:
		for key in contractions:
			value = contractions[key]
			x = x.replace(key, value)
		return x
	else:
		return x

In [ ]:
def remove_stopwords(x):
	return ' '.join([t for t in x.split() if t not in stopwords])	

def remove_special_characters(x):
	x = re.sub(r'[^\w ]+', "", x)
	x = x.replace("_", "")
	x = ' '.join(x.split())
	return x

def remove_digits(x):
	x = re.sub(r'[\d+]', "", x)
	x = ' '.join(x.split())
	return x

def remove_accented_chars(x):
	x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
	return x

In [ ]:
df.columns

Index(['reviews', 'responses'], dtype='object')

In [ ]:
df_sample['reviews'] = df_sample['reviews'].apply(lambda x: str(x).lower())
df_sample['reviews'] = df_sample['reviews'].apply(lambda x: expand_words(x))
df_sample['reviews'] = df_sample['reviews'].apply(lambda x: remove_special_characters(x))
df_sample['reviews'] = df_sample['reviews'].apply(lambda x: remove_accented_chars(x))
df_sample['reviews'] = df_sample['reviews'].apply(lambda x: remove_digits(x))

df_sample['responses'] = df_sample['responses'].apply(lambda x: str(x).lower())
df_sample['responses'] = df_sample['responses'].apply(lambda x: expand_words(x))
df_sample['responses'] = df_sample['responses'].apply(lambda x: remove_special_characters(x))
df_sample['responses'] = df_sample['responses'].apply(lambda x: remove_accented_chars(x))
df_sample['responses'] = df_sample['responses'].apply(lambda x: remove_digits(x))

## Sentiment Analysis

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 61 kB 443 kB/s 
     |████████████████████████████████| 3.3 MB 37.4 MB/s 


In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline

In [ ]:
nlp_sentiment_model = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
df_sample['sentiment'] = df_sample['reviews'].apply(lambda x : nlp_sentiment_model(x))
#df_fortraining = pd.read_csv("/content/drive/MyDrive/df_fortraining.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_sample.shape, df_sample['sentiment']

((20000, 3), 30000    [{'label': 'NEGATIVE', 'score': 0.9886061549186707}]
 30001    [{'label': 'NEGATIVE', 'score': 0.9997698664665222}]
 30002     [{'label': 'POSITIVE', 'score': 0.984809935092926}]
 30003    [{'label': 'POSITIVE', 'score': 0.9928419589996338}]
 30004    [{'label': 'POSITIVE', 'score': 0.9782479405403137}]
                                  ...                         
 49995    [{'label': 'NEGATIVE', 'score': 0.9558812975883484}]
 49996    [{'label': 'NEGATIVE', 'score': 0.9963351488113403}]
 49997    [{'label': 'NEGATIVE', 'score': 0.9996914863586426}]
 49998    [{'label': 'POSITIVE', 'score': 0.8948270678520203}]
 49999    [{'label': 'NEGATIVE', 'score': 0.9976389408111572}]
 Name: sentiment, Length: 20000, dtype: object)

In [ ]:
def sentiment_conversion(x):
  if "POSITIVE" in str(x):
    return 1
  else:
    return 0

In [ ]:
df_sample['reviews_sentiment'] = df_sample['sentiment'].apply(lambda x : sentiment_conversion(x))
df_sample['reviews_sentiment'].value_counts()

0    34241
1    15759
Name: reviews_sentiment, dtype: int64

##Another method

In [ ]:
df_sample['reviews_sentiment'] = df_sample['sentiment'].apply(lambda x : 1 if '\'POSITIVE\'' in x else 0)
df_sample['reviews_sentiment'].value_counts()

0        0
1        0
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    0
29998    0
29999    0
Name: sentiment, Length: 30000, dtype: int64

In [ ]:
df_sample['length_reviews']=df_sample['reviews'].apply(lambda x:len(x.split(" ")))
df_sample['length_responses']=df_sample['responses'].apply(lambda x:len(x.split(" ")))

In [ ]:
df_sample.columns
df_sample.isna().sum()

reviews             0
responses           0
sentiment           0
length_reviews      0
length_responses    0
dtype: int64

In [ ]:
df_sample.to_csv("/content/drive/MyDrive/ReviewsAndResponses_sentiment2.csv")